In [1]:
from IPython import display
display.clear_output()

import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.120  Python-3.11.9 torch-2.7.0+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
Setup complete  (8 CPUs, 7.9 GB RAM, 347.2/445.8 GB disk)


In [8]:
# Em apple sillicon, verificar se o MPS está disponível
import torch
print(torch.backends.mps.is_available())

False


# Treino de um modelo

In [ ]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset
# em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
# roboflow.login()

# criar um ficheiro com a key da api do roboflow, ou simplesmente substituir abaixo
#with open('api_key', "r") as file:
#    api_key = file.read().strip() 

#rf = roboflow.Roboflow(api_key)

# substituir nome do workspace e do projeto
#project = rf.workspace("dcarneiro").project("foe-bot")
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(2).download("yolov8")
# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado

In [ ]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("yolov8m.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
# results = model.train(data='FoE-bot-1', epochs=100, imgsz=640, device=[0, 1]) # intel/windows
results = model.train(data='/Users/henri/Desktop/aimbot/aimbot.v2i.yolov8/data.yaml', epochs=100, imgsz=640, device=0) # cpu & gpu

# Inferência

In [9]:
# selecionar a melhor versão do modelo fine-tuned
model = YOLO("/Users/User/AI/ia_24_25/IA/runs/detect/train/weights/best.pt")

In [1]:
import mouse
from time import sleep
import json
import math

def aim_at_best_target(json_data, screen_width=1920, screen_height=1080, model_width=640, model_height=384):
    data = json.loads(json_data)

    # Filtro apenas para labels "enemy" ou "head"
    targets = [det for det in data["detections"] if det["label"] in ["enemy", "head"]]

    if not targets:
        return  # nenhum alvo relevante

    # Encontrar o alvo com maior probabilidade
    best_target = max(targets, key=lambda d: d["prob"])

    # Coordenadas do bounding box no frame de input
    x1, y1, x2, y2 = best_target["x1"], best_target["y1"], best_target["x2"], best_target["y2"]

    # Coordenadas do centro
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    # Escalar para o tamanho real do ecrã
    scale_x = screen_width / model_width
    scale_y = screen_height / model_height

    screen_x = int(center_x * scale_x)
    screen_y = int(center_y * scale_y)

    # Mover o rato e clicar
    mouse.move(screen_x, screen_y, absolute=True, duration=0.05)
    print(f"Aiming at {best_target['label']} with prob {best_target['prob']:.2f} at ({screen_x}, {screen_y})")
    mouse.click(button='left')
    sleep(0.15)  # pequena pausa para simular ação humana

# Inferência em tempo real

In [3]:
from ultralytics import YOLO
import pyautogui
import numpy as np
import cv2
import mss
import os
import time
from scipy.spatial import distance
import pydirectinput
import ctypes

In [6]:
# Carregar o modelo
model = YOLO("/Users/7luca/Documents/University/IA/IA/best.pt")

PUL = ctypes.POINTER(ctypes.c_ulong)

class MOUSEINPUT(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class INPUT(ctypes.Structure):
    class _INPUT(ctypes.Union):
        _fields_ = [("mi", MOUSEINPUT)]
    _anonymous_ = ("_input",)
    _fields_ = [("type", ctypes.c_ulong), ("_input", _INPUT)]

# Constantes
MIN_ENEMY_SIZE = 1000
INPUT_MOUSE = 0
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_LEFTDOWN = 0x0002
MOUSEEVENTF_LEFTUP = 0x0004

def move_mouse(dx, dy):
    mi = MOUSEINPUT(dx=dx, dy=dy, mouseData=0, dwFlags=MOUSEEVENTF_MOVE, time=0, dwExtraInfo=None)
    inp = INPUT(type=INPUT_MOUSE, mi=mi)
    ctypes.windll.user32.SendInput(1, ctypes.pointer(inp), ctypes.sizeof(inp))

def click_mouse():
    extra = ctypes.c_ulong(0)
    ii_ = INPUT._INPUT()
    # Mouse down
    ii_.mi = MOUSEINPUT(0, 0, 0, MOUSEEVENTF_LEFTDOWN, 0, ctypes.pointer(extra))
    command = INPUT(type=INPUT_MOUSE, _input=ii_)
    ctypes.windll.user32.SendInput(1, ctypes.pointer(command), ctypes.sizeof(command))
    time.sleep(0.01)
    # Mouse up
    ii_.mi = MOUSEINPUT(0, 0, 0, MOUSEEVENTF_LEFTUP, 0, ctypes.pointer(extra))
    command = INPUT(type=INPUT_MOUSE, _input=ii_)
    ctypes.windll.user32.SendInput(1, ctypes.pointer(command), ctypes.sizeof(command))

def capture_screen():
    """
    Captura apenas a área do jogo no centro da tela.
    """
    monitor_width, monitor_height = pyautogui.size()
    game_width, game_height = 1366, 768

    offset_top = 0
    offset_left = 0

    top = (monitor_height // 2) - (game_height // 2) + offset_top
    left = (monitor_width // 2) - (game_width // 2) + offset_left

    with mss.mss() as sct:
        monitor = {
            "top": top,
            "left": left,
            "width": game_width,
            "height": game_height
        }
        screenshot = sct.grab(monitor)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return img, left, top, monitor["width"], monitor["height"]

def aim(x, y, sensitivity_scale):
    """
    Move o rato até ao alvo em vários passos, recalculando sempre a direção.
    """
    current_x, current_y = pyautogui.position()
    dx = int((x - current_x) * sensitivity_scale)
    dy = int((y - current_y) * sensitivity_scale)
    print(f"[AIM] dx={dx}, dy={dy}")
    move_mouse(dx, dy)

def shoot():
    click_mouse()
    time.sleep(0.05)

if __name__ == '__main__':
    os.makedirs('detections', exist_ok=True)

    screen_width, screen_height = 1366, 768
    screen_center = (screen_width // 2, screen_height // 2)

    current_target = None  # Variável para armazenar o alvo atual

    while True:
        img, offset_left, offset_top, width, height = capture_screen()
        screen_center = (width // 2, height // 2)

        results = model(img, conf=0.50)

        detections = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        centroids = []
        enemy_sizes = []

        for i, box in enumerate(detections):
            x1, y1, x2, y2 = box
            class_id = int(classes[i])
            
            # Filtrar apenas classes 2 (cabeça) e 3 (inimigo)
            if class_id in [2, 3]:
                centroid_x = int((x1 + x2) / 2)
                centroid_y = int((y1 + y2) / 2)
                centroids.append((centroid_x, centroid_y, class_id))
                # Calcular tamanho do inimigo (área do bounding box)
                size = (x2 - x1) * (y2 - y1)
                enemy_sizes.append(size)

        # Priorizar cabeça (classe 2), senão inimigo (classe 3)
        headshots = [c for c in centroids if c[2] == 2]
        bodies = [c for c in centroids if c[2] == 3]

        target = None
        if headshots:
            target = min(headshots, key=lambda c: distance.euclidean(screen_center, (c[0], c[1])))
        elif bodies:
            target = min(bodies, key=lambda c: distance.euclidean(screen_center, (c[0], c[1])))

        # Verificar se o alvo atual ainda está presente
        if current_target and current_target in centroids:
            target = current_target  # Continuar mirando no alvo atual
        else:
            current_target = target  # Atualizar o alvo atual

        # --- NOVO: Andar para a frente se o inimigo for pequeno ---
        should_walk = False
        if enemy_sizes:
            min_enemy_size = min(enemy_sizes)
            if min_enemy_size < MIN_ENEMY_SIZE:
                should_walk = True

        if should_walk:
            print("A andar para a frente (inimigo pequeno detectado)")
            pydirectinput.keyDown('w')
        else:
            pydirectinput.keyUp('w')

        if target:
            local_x, local_y = target[0], target[1]
            absolute_x = offset_left + local_x
            absolute_y = offset_top + local_y

            print(f"[+] Aiming at local: ({local_x},{local_y}), absoluto: ({absolute_x},{absolute_y})")

            aim(absolute_x, absolute_y, sensitivity_scale=2.6)
            time.sleep(0.05)
            shoot()

            cv2.line(img, screen_center, (local_x, local_y), (0, 0, 255), 2)

        annotated_frame = results[0].plot()
        combined_frame = annotated_frame.copy()

        if target:
            cv2.line(combined_frame, screen_center, (local_x, local_y), (0, 0, 255), 2)

        result_path = os.path.join('detections', f"result_{int(time.time())}.jpg")
        cv2.imwrite(result_path, combined_frame)
        print(f"Frame salvo em: {result_path}")
# ...existing code...


0: 384x640 (no detections), 232.1ms
Speed: 2.4ms preprocess, 232.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Frame salvo em: detections\result_1747416311.jpg

0: 384x640 (no detections), 222.5ms
Speed: 2.3ms preprocess, 222.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Frame salvo em: detections\result_1747416312.jpg

0: 384x640 (no detections), 228.4ms
Speed: 1.8ms preprocess, 228.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Frame salvo em: detections\result_1747416312.jpg

0: 384x640 (no detections), 223.0ms
Speed: 1.7ms preprocess, 223.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Frame salvo em: detections\result_1747416312.jpg

0: 384x640 (no detections), 226.4ms
Speed: 1.9ms preprocess, 226.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Frame salvo em: detections\result_1747416313.jpg

0: 384x640 (no detections), 218.0ms
Speed: 2.0ms preprocess, 218.0ms inference,

KeyboardInterrupt: 